In [8]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2025-12-03 17:08:50--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
18.161.245.40, 18.161.245.54, 18.161.245.3, ...a8wq0iu5.cloudfront.net)... 
connected. to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.161.245.40|:80... 
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa’

phix.fa             100%[===================>]   5.40K  --.-KB/s    in 0s      

2025-12-03 17:08:50 (315 MB/s) - ‘phix.fa’ saved [5528/5528]



In [9]:
#reading the genome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == ">":
                genome += line.rstrip()
    return genome

In [12]:
#reading the genome file which we just downloaded
genome = readGenome('phix.fa')

In [13]:
#Naive exact matching algorithm
def naive(p,t):
    occurences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if not t[i+j] == p[j]:
                match = False
                break
        if match:
            occurences.append(i)
    return occurences

In [14]:
t = 'AGCTTAGATAGC'
p = 'AG'
naive(p,t)

[0, 5, 9]

In [15]:
import random 
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome) - readLen) - 1
        reads.append(genome[start : start + readLen])
    return reads

In [16]:
#lets genrate the random reads from the genome we downloaded eariler by taking subsequences of a genome string taking from a random position
#Example with artificial reads
reads = generateReads(genome, 100, 100)

numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
print(f"{numMatched} / {len(reads)} reads matched exactly")


100 / 100 reads matched exactly


In [ ]:
#lets try this same example with real data.
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

In [21]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

phix_reads = readFastq('ERR266411_1.first1000.fastq')

numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print(f"{numMatched} / {n} reads matched the genome exactly")
    

7 / 1000 reads matched the genome exactly


In [53]:
''' These are sequecing reads actually from the genome and yet 7 out of 1000 actually matched!
why?? 
1) sequencing errors
2)differences b/w the sequences and reference genome
3) what if the reads are coming from reverse strand of the genome
'''
#What if we take a part of a read an match with the genome? maybe 30 bases instead of 100

numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print(f"{numMatched} / {n} reads matched the genome exactly")
    

459 / 1000 reads matched the genome exactly


In [64]:
#reverse complement of the genome
def reverseComplement(s):
    complement = {'A':'T', 
                  'T':'A', 
                  'G':'C',
                  'C':'G',
                  'N':'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t
    
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print(f"{numMatched} / {n} reads matched the genome exactly")    

932 / 1000 reads matched the genome exactly
